In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt


# Define Functions, File Paths, and Parameters

In [2]:
def getValidityCPS2(df):
    df['validity'] = 1
    df['validity'] = np.where( ( df['LeftPupilValidity'] == 'Invalid') & (df['RightPupilValidity'] == 'Invalid'), 0, df['validity'] )
    df['validity'] = np.where( ( df['LeftPupilValidity'] == 'Valid') & (df['RightPupilValidity'] == 'Invalid'), 0.3, df['validity'] )
    df['validity'] = np.where( ( df['LeftPupilValidity'] == 'Invalid') & (df['RightPupilValidity'] == 'Valid'), 0.7, df['validity'] )
    return df['validity']

In [3]:
MASTERFILE = "/Users/angelina/Dropbox (Emotive Computing)/CPS_Gaze_Fixations/utils/CPS2_Gaze_All_Participants_Masterfile.csv"
OUTPUTDIR = '/Users/angelina/Dropbox (Emotive Computing)/CPS_Gaze_Fixations/RQA/'
PARTICIPANT_BLOCK_FULL_TIME_SERIES = OUTPUTDIR + 'Participant_Block_Full_Time_Series/'
PARTICIPANT_BLOCK_225ms_TIME_SERIES = OUTPUTDIR + 'Participant_Block_225ms_Time_Series/'


In [4]:
# Time series parameters
ms_in_block = 900000 # ms
time_step = 225      # ms
num_bins = int(ms_in_block / time_step)
time_step_labels = np.arange(0, ms_in_block+time_step, time_step)   # label for start ms of time step

# Screen grid parameters
screen_width = 1920
screen_height = 1080
num_cols = 10
num_rows = 10
col_width = screen_width/num_cols
row_height = screen_height/num_rows

# Create array to hold name ID of all screen-grid locations
grid_locations = np.arange(1, (num_rows * num_cols) + 1)   # (1, 101)
grid_locations = np.reshape(grid_locations, (num_rows, num_cols))  # (10,10)
print(grid_locations)

# DataFrame for all time steps to merge w/ participant and block's DataFrame
participant_block_time_steps = pd.DataFrame()
participant_block_time_steps['time_step_start'] = np.arange(0, ms_in_block, time_step)
participant_block_time_steps['time_step_end'] = np.arange(time_step, ms_in_block+time_step, time_step)
display(participant_block_time_steps)

# columns of participant and block's DataFrame to group by
cols_for_grp = ["school", "team", "GROUPID", "block", "pid", "time_step_start", "time_step_end"]


[[  1   2   3   4   5   6   7   8   9  10]
 [ 11  12  13  14  15  16  17  18  19  20]
 [ 21  22  23  24  25  26  27  28  29  30]
 [ 31  32  33  34  35  36  37  38  39  40]
 [ 41  42  43  44  45  46  47  48  49  50]
 [ 51  52  53  54  55  56  57  58  59  60]
 [ 61  62  63  64  65  66  67  68  69  70]
 [ 71  72  73  74  75  76  77  78  79  80]
 [ 81  82  83  84  85  86  87  88  89  90]
 [ 91  92  93  94  95  96  97  98  99 100]]


,time_step_start,time_step_end
0,0,225
1,225,450
2,450,675
3,675,900
4,900,1125
...,...,...
3995,898875,899100
3996,899100,899325
3997,899325,899550
3998,899550,899775


# Import Masterfile

In [5]:
# Read in masterfile which contains one row per participant + block and filepaths to each file we need
dfMaster = pd.read_csv(MASTERFILE)

dfMaster['Analyze'] = np.where(dfMaster['ParticipantBlock'] == "CPS2-CU-T60-PB-Transfer", "no", dfMaster['Analyze'])
dfMaster['Analyze'] = np.where(dfMaster['ParticipantBlock'] == "CPS2-CU-T29-PC-Transfer", "no", dfMaster['Analyze'])
dfMaster['Analyze'] = np.where(dfMaster['ParticipantBlock'] == "CPS2-CU-T27-PA-Transfer", "no", dfMaster['Analyze'])

dfMasterSelected = dfMaster.loc[dfMaster['Analyze'] == 'yes']
dfMasterSelected.reset_index(drop=True, inplace=True)

# Informational
print("Number of unique team + blocks in Masterfile: ", len(dfMasterSelected['GROUPID_block'].unique()))
print("Number of participant + blocks in Masterfile: ", len(dfMasterSelected))


Number of unique team + blocks in Masterfile:  582
Number of participant + blocks in Masterfile:  1516


# Loop through all files for each participant and block

In [6]:


# For every participant and block
for i in range(0, dfMasterSelected.shape[0]): # (146, 147), [146, 380, 1509], (258,259) - horizontal zoom pos
    master_row = dfMasterSelected.iloc[i]
    print("Processing %d: %s" % (i+1, master_row['ParticipantBlock']))

    dfParticipantBlockRawCoords = pd.DataFrame()

    ### 1. Read in the data files we need ###
    
    # Raw Gaze of current participant and block
    dfRaw = pd.read_csv(master_row['FullPath_gaze'], sep="\t")
    dfRaw['timestamp'] = (dfRaw['DeviceTime'] - dfRaw['DeviceTime'].iloc[0]) / 1000  # in ms
    dfRaw['validity'] = getValidityCPS2(dfRaw)
    dfRaw['Raw_x'] = (dfRaw['LeftGazePointXMonitor'] + dfRaw['RightGazePointXMonitor']) / float(2)
    dfRaw['Raw_y'] = (dfRaw['LeftGazePointYMonitor'] + dfRaw['RightGazePointYMonitor']) / float(2)
    dfRaw['GROUPID'] = master_row['GROUPID']
    dfRaw['block'] = master_row['Block']
    dfRaw['pid'] = master_row['pid']
    # Add participant and block's raw gaze to their df
    raw_cols = ['school', 'team', 'pid', 'GROUPID', 'block', 'timestamp', 'validity', 'Raw_x', 'Raw_y']
    dfParticipantBlockRawCoords = pd.concat([dfParticipantBlockRawCoords, dfRaw.loc[(dfRaw['timestamp'] >= 0) & \
                                                                                    (dfRaw['timestamp'] <= ms_in_block), \
                                                                                    raw_cols]], ignore_index=True)
    
    
    ### 2. Categorize coordinates into bins ###
    
    # Categorize raw gaze coordinates into bins
    dfParticipantBlockRawCoords['time_step_start'] = pd.cut(dfParticipantBlockRawCoords['timestamp'], \
                                                            bins=time_step_labels,  labels=time_step_labels[:-1], right=False)
    dfParticipantBlockRawCoords['time_step_end'] = pd.cut(dfParticipantBlockRawCoords['timestamp'], \
                                                          bins=time_step_labels, labels=time_step_labels[1:], right=False)
    # Update column types from categorical to int
    dfParticipantBlockRawCoords = dfParticipantBlockRawCoords.astype({"time_step_start": int, "time_step_end": int})

 

    ### 3. Average X and Y for each bin
    
    # columns of participant and block's DataFrame to group by
    cols_for_grp = ["school", "team", "GROUPID", "block", "pid", "time_step_start", "time_step_end"]
    
    # Raw coordinates
    grp = dfParticipantBlockRawCoords.groupby(cols_for_grp, as_index=False)
    agg = grp.agg({'Raw_x': np.nanmean, 'Raw_y': np.nanmean})
    dfParticipantBlockRawCoordsTimeSeries = pd.merge(participant_block_time_steps, agg, \
                                                     on=['time_step_start', 'time_step_end'], how='left')


    ### 4. IMPUTE DATA ###
    dfParticipantBlockRawCoordsImp = dfParticipantBlockRawCoordsTimeSeries.loc[:, ['Raw_x', 'Raw_y']].interpolate(method='spline', order=2)

    dfParticipantBlockRawCoordsTimeSeries[['Raw_x_imp', 'Raw_y_imp']] = dfParticipantBlockRawCoordsImp

       
    
    ### 5. Assign grid location for each time step ###
    
    # Assign grid location of raw coordinates for each time step
    dfParticipantBlockRawCoordsTimeSeries['row_index'] = np.ceil(dfParticipantBlockRawCoordsTimeSeries['Raw_y_imp'] / row_height) - 1  # subtract 1 because result is not a zero-index
    dfParticipantBlockRawCoordsTimeSeries['col_index'] = np.ceil(dfParticipantBlockRawCoordsTimeSeries['Raw_x_imp'] / col_width) - 1   # subtract 1 because result is not a zero-index
    dfParticipantBlockRawCoordsTimeSeries['in_bounds'] = np.where((dfParticipantBlockRawCoordsTimeSeries['row_index'] >= 0) & \
                                                        (dfParticipantBlockRawCoordsTimeSeries['col_index'] >= 0) & \
                                                        (dfParticipantBlockRawCoordsTimeSeries['row_index'] < num_rows) & \
                                                        (dfParticipantBlockRawCoordsTimeSeries['col_index'] < num_cols), 1, 0)

    for i,row in dfParticipantBlockRawCoordsTimeSeries.iterrows():
        if row['in_bounds'] == 1:
            dfParticipantBlockRawCoordsTimeSeries.loc[i, 'grid_loc'] = grid_locations[int(row['row_index']), int(row['col_index'])]
        elif row['in_bounds'] == 0:
            dfParticipantBlockRawCoordsTimeSeries.loc[i, 'grid_loc'] = 0

    

    ### 6. Save participant and block sampled (225 ms) time series as csv ###
    filename = str(master_row['GROUPID']) + "-" + master_row['Block'] + "-" + master_row['pid'] + "-225ms-"
    dfParticipantBlockRawCoordsTimeSeries.to_csv(PARTICIPANT_BLOCK_225ms_TIME_SERIES + filename + "RAW.csv", index=False)
    
    ### 7. Add samples (225ms) time series file path to Masterfile ###
    
    # Raw coordinates
    dfMaster.loc[(dfMaster['GROUPID'] == master_row['GROUPID']) & \
                 (dfMaster['Block'] == master_row['Block']) & \
                 (dfMaster['pid'] == master_row['pid']), '225ms-time-series-RAW'] \
                  = filename + "RAW.csv"

    
print("File Generation Complete")   


Processing 1: CPS2-CU-T63-PB-ExpBlock1


/Users/angelina/miniforge3/envs/env_tf/lib/python3.9/site-packages/scipy/interpolate/_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


Processing 2: CPS2-CU-T16-PA-ExpBlock2
Processing 3: CPS2-CU-T36-PC-Transfer
Processing 4: CPS2-ASU-T67-PC-Warmup
Processing 5: CPS2-ASU-T80-PC-ExpBlock1
Processing 6: CPS2-ASU-T86-PD-ExpBlock1
Processing 7: CPS2-ASU-T42-PC-ExpBlock1
Processing 8: CPS2-CU-T33-PB-Transfer


/Users/angelina/miniforge3/envs/env_tf/lib/python3.9/site-packages/scipy/interpolate/_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


Processing 9: CPS2-ASU-T60-PD-ExpBlock2


...

Processing 1506: CPS2-ASU-T66-PA-ExpBlock1
Processing 1507: CPS2-ASU-T13-PB-ExpBlock2
Processing 1508: CPS2-CU-T16-PC-ExpBlock1


/Users/angelina/miniforge3/envs/env_tf/lib/python3.9/site-packages/scipy/interpolate/_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


Processing 1509: CPS2-CU-T65-PC-Warmup


/Users/angelina/miniforge3/envs/env_tf/lib/python3.9/site-packages/scipy/interpolate/_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


Processing 1510: CPS2-CU-T13-PA-Transfer
Processing 1511: CPS2-CU-T32-PC-ExpBlock2
Processing 1512: CPS2-ASU-T37-PB-ExpBlock1
Processing 1513: CPS2-ASU-T51-PC-Transfer
Processing 1514: CPS2-ASU-T97-PB-Warmup
Processing 1515: CPS2-ASU-T20-PA-Warmup
Processing 1516: CPS2-CU-T46-PA-Warmup
File Generation Complete


In [7]:
# Save new Masterfile to csv
filename = OUTPUTDIR + "CPS2_Gaze_All_Participants_Masterfile_TimeSeries.csv"
dfMaster.to_csv(filename, index=False)
print("%s generated." % (filename))


/Users/angelina/Dropbox (Emotive Computing)/CPS_Gaze_Fixations/RQA/CPS2_Gaze_All_Participants_Masterfile_TimeSeries.csv generated.
